In [1]:
import pandas as pd
import numpy as np
from pyprojroot import here

from language_tool_python import LanguageTool

In [2]:
train_df = pd.read_csv(here('commonlit-evaluate-student-summaries/summaries_train.csv'))
train_promts_df = pd.read_csv(here('commonlit-evaluate-student-summaries/prompts_train.csv'))

In [5]:
train_df

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [33]:
pd.DataFrame(
            columns = ['TYPOS', 'TYPOGRAPHY', 'CASING', 'PUNCTUATION', 'GRAMMAR', 'REDUNDANCY', 'STYLE', 'CONFUSED_WORDS', 'OTHER', 'TOTAL_ERRORS'],

            data = np.zeros((1, 10)).astype(int)
        )

,TYPOS,TYPOGRAPHY,CASING,PUNCTUATION,GRAMMAR,REDUNDANCY,STYLE,CONFUSED_WORDS,OTHER,TOTAL_ERRORS
0,0,0,0,0,0,0,0,0,0,0


In [44]:
def grammar_check(df):
    grammar_tool = LanguageTool('en-US')
    grammar_df_cols = [
        'TYPOS', 'TYPOGRAPHY', 'CASING', 'PUNCTUATION', 'GRAMMAR', 'REDUNDANCY', 'STYLE', 'CONFUSED_WORDS', 'OTHER', 'TOTAL_ERRORS', 
        'MIN_ERROR_LOCATION', 'MAX_ERROR_LOCATION', 'AVERAGE_ERROR_LOCATION'
    ]
    grammar_features_df = pd.DataFrame(columns = grammar_df_cols)
    
    for i, row in df.iterrows():
        new_row = pd.DataFrame(
            columns = grammar_df_cols,
            data = np.zeros((1, len(grammar_df_cols))).astype(int),
            index = [i]
        )
        text = row['text']
        matches = grammar_tool.check(text)
        new_row['TOTAL_ERRORS'] = len(matches)
        
        error_locations = []
        for match in matches:
            if match.category in new_row.columns:
                new_row[match.category] += 1
                error_locations.append(match.offset)
            else:
                new_row['OTHER'] += 1
                error_locations.append(match.offset)
        
        if len(error_locations) == 0:
            pass
        
        else:
            new_row['MIN_ERROR_LOCATION'] = min(error_locations)
            new_row['MAX_ERROR_LOCATION'] = max(error_locations)
            new_row['AVERAGE_ERROR_LOCATION'] = np.mean(error_locations)
        
        grammar_features_df = pd.concat([grammar_features_df, new_row], ignore_index = False)
        
    return grammar_features_df

In [45]:
grammar_features_df = grammar_check(train_df.head())

In [46]:
grammar_features_df

,TYPOS,TYPOGRAPHY,CASING,PUNCTUATION,GRAMMAR,REDUNDANCY,STYLE,CONFUSED_WORDS,OTHER,TOTAL_ERRORS,MIN_ERROR_LOCATION,MAX_ERROR_LOCATION,AVERAGE_ERROR_LOCATION
0,3,0,0,0,0,0,0,0,0,3,22,263,165.000000
1,1,0,0,1,0,1,0,0,0,3,64,177,109.666667
2,1,1,0,0,0,0,0,0,0,2,256,1282,769.000000
3,1,2,0,0,0,0,0,0,0,3,54,96,79.000000
4,14,1,0,0,0,0,0,0,0,15,24,1121,537.800000


In [ ]:
grammar_features_df

In [17]:
np.zeros((1,10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [13]:
pd.Series(categories).value_counts()

TYPOS                     13553
TYPOGRAPHY                 4719
CASING                     2065
PUNCTUATION                1994
GRAMMAR                    1056
REDUNDANCY                  436
MISC                        375
STYLE                       283
CONFUSED_WORDS              247
COLLOCATIONS                 27
COMPOUNDING                  25
AMERICAN_ENGLISH_STYLE       11
NONSTANDARD_PHRASES           8
SEMANTICS                     1
Name: count, dtype: int64

In [8]:
list = []

In [9]:
len(list)

0

In [10]:
grammar_tool = LanguageTool('en-US')
messages = []
categories = []
for i, row in train_df.iterrows():
    matches = grammar_tool.check(row['text'])
    if len(matches) > 0:
        for match in matches:
            messages.append(match.message)
            categories.append(match.category)
            

In [4]:
tool.close()